In [4]:
import pandas as pd
import requests
import json
import os


In [5]:
response = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')
response.status_code

200

In [6]:
response.headers['content-type']

'text/tab-separated-values; charset=utf-8'

In [7]:
response.headers

{'Content-Type': 'text/tab-separated-values; charset=utf-8', 'Content-Length': '335079', 'Connection': 'keep-alive', 'Last-Modified': 'Fri, 25 Aug 2017 07:33:02 GMT', 'Accept-Ranges': 'bytes', 'Server': 'AmazonS3', 'Date': 'Thu, 02 Jun 2022 08:59:39 GMT', 'ETag': '"64f3fc4cfaa55b6d1f34181844f66f2f"', 'X-Cache': 'Hit from cloudfront', 'Via': '1.1 de7bb21ade87bd155cf024cdea5bf62e.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'LHR50-P2', 'X-Amz-Cf-Id': 'GeESc0Ykea5SaFLv0jIYczw9b5OMZW2ql4hJN35pNq-Wk25iaLmQZw==', 'Age': '24220'}

In [8]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
with open(os.path.join(url.split('/')[-1]), mode='wb') as file:
        file.write(response.content)

In [9]:
image_predictions = pd.read_csv('image-predictions.tsv', sep='\t')

In [10]:
import tweepy

In [11]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

In [15]:
# query the Twitter API for each tweet's JSON data using Python's Tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit = True)

In [16]:
twitter_feed = pd.read_csv('twitter-archive-enhanced.csv', sep =',')

In [17]:
twitter_feed.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [18]:
tweet_ids = twitter_feed.tweet_id

In [25]:
# this script runs in 33 mins
not_retrieved_id = []
with open('tweet_json.txt', 'w', encoding='UTF-8') as json_file:
    for id_ in tweet_ids:
        try:
            tweet = api.get_status(id_)
            tweet = tweet._json
            json.dump(tweet, json_file)
            json_file.write('\n')
        except:
            print('Failed retrieving tweet with id ', id_)
            not_retrieved_id.append(id_)

Failed retrieving tweet with id  888202515573088257
Failed retrieving tweet with id  873697596434513921
Failed retrieving tweet with id  872668790621863937
Failed retrieving tweet with id  872261713294495745
Failed retrieving tweet with id  869988702071779329
Failed retrieving tweet with id  866816280283807744
Failed retrieving tweet with id  861769973181624320
Failed retrieving tweet with id  856602993587888130
Failed retrieving tweet with id  856330835276025856
Failed retrieving tweet with id  851953902622658560
Failed retrieving tweet with id  851861385021730816
Failed retrieving tweet with id  845459076796616705
Failed retrieving tweet with id  844704788403113984
Failed retrieving tweet with id  842892208864923648
Failed retrieving tweet with id  837366284874571778
Failed retrieving tweet with id  837012587749474308
Failed retrieving tweet with id  829374341691346946
Failed retrieving tweet with id  827228250799742977
Failed retrieving tweet with id  812747805718642688
Failed retri

Rate limit reached. Sleeping for: 642


Failed retrieving tweet with id  754011816964026368
Failed retrieving tweet with id  680055455951884288


Rate limit reached. Sleeping for: 635


In [26]:
df = []
with open('tweet_json.txt') as f:
    for line in f:
        tweet = (json.loads(line))
        tweet_id = tweet['id']
        retweet_count = tweet['retweet_count']
        favorite_count = tweet['favorite_count']
        create_date = tweet['created_at']
        df.append({'retweet_count' : retweet_count,
                  'favorite_count' : favorite_count,
                  'create_date' : create_date,
                  'tweet_id' : tweet_id})
        
fave_retweet_data = pd.DataFrame(df, columns = ['tweet_id', 'retweet_count', 'favorite_count', 'create_date'])

In [27]:
fave_retweet_data.head()

,tweet_id,retweet_count,favorite_count,create_date
0,892420643555336193,7027,33877,Tue Aug 01 16:23:56 +0000 2017
1,892177421306343426,5305,29380,Tue Aug 01 00:17:27 +0000 2017
2,891815181378084864,3489,22099,Mon Jul 31 00:18:03 +0000 2017
3,891689557279858688,7239,36995,Sun Jul 30 15:58:51 +0000 2017
4,891327558926688256,7779,35372,Sat Jul 29 16:00:24 +0000 2017
